In [1]:
pip install rasterio cupy-cuda11x

   ---------------------------------------- 0.0/76.9 MB ? eta -:--:--
    --------------------------------------- 1.3/76.9 MB 7.0 MB/s eta 0:00:11
   - -------------------------------------- 2.6/76.9 MB 6.9 MB/s eta 0:00:11
   -- ------------------------------------- 4.2/76.9 MB 6.9 MB/s eta 0:00:11
   -- ------------------------------------- 5.8/76.9 MB 7.1 MB/s eta 0:00:11
   --- ------------------------------------ 7.3/76.9 MB 7.1 MB/s eta 0:00:10
   ---- ----------------------------------- 8.9/76.9 MB 7.2 MB/s eta 0:00:10
   ----- ---------------------------------- 10.5/76.9 MB 7.3 MB/s eta 0:00:10
   ------ --------------------------------- 12.6/76.9 MB 7.7 MB/s eta 0:00:09
   ------- -------------------------------- 14.7/76.9 MB 8.1 MB/s eta 0:00:08
   -------- ------------------------------- 17.3/76.9 MB 8.5 MB/s eta 0:00:08
   ---------- ----------------------------- 19.7/76.9 MB 8.8 MB/s eta 0:00:07
   ---------- ----------------------------- 20.2/76.9 MB 8.2 MB/s eta 0:00:07


In [4]:
import cupy
print(cupy.__version__)



13.4.1


In [5]:
import cupy as cp
print(cp.get_device_count())  # 사용 가능한 GPU 디바이스 수 확인


AttributeError: module 'cupy' has no attribute 'get_device_count'

In [7]:
#제주,고산,서귀포_PVgen데이터합성
import os
import re
import numpy as np
import rasterio
from rasterio.transform import from_origin
import xml.etree.ElementTree as ET

# 입력 및 출력 디렉토리 설정
dirs = {
    "jeju": r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju",
    "gosan": r"E:\2025_Jeju_SectorCoupling\PVgen_Gosan",
    "seoguipo": r"E:\2025_Jeju_SectorCoupling\PVgen_Seoguipo",
}
output_dir = r"E:\2025_Jeju_SectorCoupling\PVgen"
os.makedirs(output_dir, exist_ok=True)

# 래스터 파일 정렬
files = {k: sorted([f for f in os.listdir(v) if f.endswith(".tif")]) for k, v in dirs.items()}

# 파일 개수 검증
if len(set(map(len, files.values()))) != 1:
    raise ValueError("각 디렉토리에 동일한 개수의 래스터 파일이 있어야 합니다.")

# 파일명 정리 함수
def remove_prefix_before_date(filename):
    return re.sub(r"^.*?(\d{4}-\d{2}-\d{2}.*)", r"\1", filename)

# TFW 파일 생성 함수
def save_tfw(transform, tfw_path):
    """World 파일(.tfw) 저장"""
    with open(tfw_path, 'w') as f:
        f.write(f"{transform.a:.10f}\n")  # X pixel size
        f.write(f"{transform.b:.10f}\n")  # Rotation (usually 0)
        f.write(f"{transform.d:.10f}\n")  # Rotation (usually 0)
        f.write(f"{transform.e:.10f}\n")  # Y pixel size (negative)
        f.write(f"{transform.xoff:.10f}\n")  # X coordinate of top left pixel
        f.write(f"{transform.yoff:.10f}\n")  # Y coordinate of top left pixel

# XML 메타데이터 저장 함수
def save_xml(metadata, xml_path):
    """XML 메타데이터 저장"""
    root = ET.Element("RasterMetadata")
    ET.SubElement(root, "CRS").text = metadata["crs"]
    ET.SubElement(root, "NoDataValue").text = str(metadata["nodata_value"])
    
    transform = ET.SubElement(root, "Transform")
    ET.SubElement(transform, "PixelSizeX").text = str(metadata["transform"][0])
    ET.SubElement(transform, "PixelSizeY").text = str(metadata["transform"][4])
    ET.SubElement(transform, "UpperLeftX").text = str(metadata["transform"][2])
    ET.SubElement(transform, "UpperLeftY").text = str(metadata["transform"][5])
    
    source_files = ET.SubElement(root, "SourceFiles")
    for src in metadata["source_files"]:
        ET.SubElement(source_files, "File").text = src

    tree = ET.ElementTree(root)
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

# 래스터 병합 및 저장
for f1, f2, f3 in zip(files["jeju"], files["gosan"], files["seoguipo"]):
    raster_paths = {k: os.path.join(dirs[k], f) for k, f in zip(dirs.keys(), [f1, f2, f3])}
    
    # TIFF 파일 불러오기 및 NumPy 배열 변환
    rasters = {k: rasterio.open(path) for k, path in raster_paths.items()}
    arrays = {k: rasters[k].read(1).astype(np.float32) for k in rasters}

    # NoData(Null) 값을 0으로 변환 후 합산
    result_arr = sum(np.nan_to_num(arr, nan=0) for arr in arrays.values())

    # 좌표 변환 정보 유지
    ref_raster = rasters["jeju"]
    transform = ref_raster.transform
    profile = ref_raster.profile.copy()
    
    # 출력 파일 경로 설정
    output_filename = f"TotalSolargen_{remove_prefix_before_date(f1)}"
    output_tif = os.path.join(output_dir, output_filename + ".tif")
    output_tfw = os.path.join(output_dir, output_filename + ".tfw")
    output_xml = os.path.join(output_dir, output_filename + ".xml")
    
    # TIFF 저장
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(result_arr, 1)

    # TFW 파일 저장
    save_tfw(transform, output_tfw)

    # XML 메타데이터 저장
    metadata = {
        "source_files": [f1, f2, f3],
        "transform": transform.to_gdal(),  # GDAL 형식 변환
        "crs": str(ref_raster.crs),
        "nodata_value": profile.get("nodata", None)
    }
    save_xml(metadata, output_xml)
    
    print(f"결과 저장: {output_tif}")
    print(f"TFW 저장: {output_tfw}")
    print(f"XML 메타데이터 저장: {output_xml}")

print("모든 래스터 병합이 완료되었습니다.")

os.makedirs(output_dir, exist_ok=True)

# 래스터 파일 정렬
files = {k: sorted(f for f in os.listdir(v) if f.endswith(".tif")) for k, v in dirs.items()}
if len(set(map(len, files.values()))) != 1:
    raise ValueError("각 디렉토리에 동일한 개수의 래스터 파일이 있어야 합니다.")

# 파일명 정리 함수
def clean_filename(filename):
    return re.sub(r"^.*?(\d{4}-\d{2}-\d{2}.*)", r"\1", filename)

# TFW 및 XML 저장 함수
def save_tfw(transform, path):
    with open(path, 'w') as f:
        f.writelines(f"{v:.10f}\n" for v in [transform.a, transform.b, transform.d, transform.e, transform.xoff, transform.yoff])

def save_xml(metadata, path):
    root = ET.Element("RasterMetadata")
    ET.SubElement(root, "CRS").text = metadata["crs"]
    ET.SubElement(root, "NoDataValue").text = str(metadata["nodata"])
    transform = ET.SubElement(root, "Transform")
    for k, v in zip(["PixelSizeX", "PixelSizeY", "UpperLeftX", "UpperLeftY"], metadata["transform"][:6:2]):
        ET.SubElement(transform, k).text = str(v)
    sources = ET.SubElement(root, "SourceFiles")
    [ET.SubElement(sources, "File").text for f in metadata["source_files"]]
    ET.ElementTree(root).write(path, encoding="utf-8", xml_declaration=True)

# 래스터 병합 및 저장
for f1, f2, f3 in zip(*files.values()):
    raster_paths = {k: os.path.join(dirs[k], f) for k, f in zip(dirs, [f1, f2, f3])}
    with rasterio.open(raster_paths["jeju"]) as ref:
        transform, profile = ref.transform, ref.profile.copy()
        arrays = [np.nan_to_num(rasterio.open(r).read(1), nan=0) for r in raster_paths.values()]
        result_arr = sum(arrays)
    
    output_name = f"TotalSolargen_{clean_filename(f1)}"
    output_paths = {ext: os.path.join(output_dir, output_name + ext) for ext in [".tif", ".tfw", ".xml"]}
    
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')
    with rasterio.open(output_paths[".tif"], 'w', **profile) as dst:
        dst.write(result_arr, 1)
    
    save_tfw(transform, output_paths[".tfw"])
    save_xml({"source_files": [f1, f2, f3], "transform": transform.to_gdal(), "crs": str(ref.crs), "nodata": profile.get("nodata", None)}, output_paths[".xml"])
    
    print(f"저장 완료: {output_paths}")

print("모든 래스터 병합 완료.")



결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 13-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 14-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.tfw
XML 메타데이터 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 15-00-00+09-00.tif.xml
결과 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 16-00-00+09-00.tif.tif
TFW 저장: E:\2025_Jeju_SectorCoupling\PVgen\TotalSolargen_2024-09-08 16-00-00+09-00.tif.

KeyboardInterrupt: 

In [8]:
pip install geopandas shapely fiona oneapi


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement oneapi (from versions: none)
ERROR: No matching distribution found for oneapi


In [1]:
pip install rasterstats

Note: you may need to restart the kernel to use updated packages.


In [14]:
import geopandas as gpd
from shapely.geometry import Polygon
import warnings

# 경고 무시 설정 (유니코드 오류로 인한 경고)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def read_shapefile_with_encoding(input_shapefile):
    # 시도할 인코딩 리스트
    encodings = ['euc-kr', 'cp949', 'ISO-8859-1', 'utf-8']
    
    # 인코딩 순차적으로 시도
    for encoding in encodings:
        try:
            print(f"시도 중: {encoding}")
            gdf = gpd.read_file(input_shapefile, encoding=encoding)
            return gdf
        except UnicodeDecodeError as e:
            print(f"인코딩 오류 발생 ({encoding}): {e}")
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
    
    # 만약 모든 인코딩이 실패할 경우
    raise ValueError("모든 인코딩 시도에서 실패했습니다. 파일을 확인해주세요.")

# 입력 파일 및 출력 파일 경로
input_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_resi.shp"
output_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp"
buffer_distance = 100  # 100 미터

# 인코딩 오류 처리 후 Shapefile 읽기
try:
    gdf = read_shapefile_with_encoding(input_shapefile)
    
    # 좌표계가 미터로 설정되어 있는지 확인하고, 아니면 변환
    if gdf.crs != 'EPSG:3395':  # EPSG:3395는 미터 단위의 CRS
        gdf = gdf.to_crs(epsg=3395)

    # 버퍼 생성 (단위는 미터)
    gdf['buffer'] = gdf.geometry.buffer(buffer_distance)

    # 결과를 새로운 Shapefile로 저장
    gdf[['buffer']].to_file(output_shapefile)

    print(f"버퍼가 {output_shapefile}에 생성되었습니다.")

except ValueError as e:
    print(f"Shapefile을 읽을 수 없습니다: {e}")



시도 중: euc-kr
버퍼가 E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp에 생성되었습니다.


In [15]:
#merge
import geopandas as gpd
import pandas as pd
import warnings

# 경고 무시 설정 (유니코드 오류로 인한 경고)
warnings.filterwarnings("ignore", category=RuntimeWarning)

def read_shapefile_with_encoding(input_shapefile):
    # 시도할 인코딩 리스트
    encodings = ['euc-kr', 'cp949', 'ISO-8859-1', 'utf-8']
    
    # 인코딩 순차적으로 시도
    for encoding in encodings:
        try:
            print(f"시도 중: {encoding}")
            gdf = gpd.read_file(input_shapefile, encoding=encoding)
            return gdf
        except UnicodeDecodeError as e:
            print(f"인코딩 오류 발생 ({encoding}): {e}")
        except Exception as e:
            print(f"알 수 없는 오류 발생: {e}")
    
    # 만약 모든 인코딩이 실패할 경우
    raise ValueError("모든 인코딩 시도에서 실패했습니다. 파일을 확인해주세요.")

# 입력 파일 경로 설정 (3개의 Shapefile)
input_shapefile_1 = r"E:\2025_Jeju_SectorCoupling\JejuNationalPark.shp"
input_shapefile_2 = r"E:\2025_Jeju_SectorCoupling\Jeju_Masking_buffer.shp"
input_shapefile_3 = r"E:\2025_Jeju_SectorCoupling\TL_SPRD_RW_50000.shp"
# 출력 파일 경로 설정
output_shapefile = r"E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp"

# Shapefile 읽기
gdf1 = read_shapefile_with_encoding(input_shapefile_1)
gdf2 = read_shapefile_with_encoding(input_shapefile_2)
gdf3 = read_shapefile_with_encoding(input_shapefile_3)

# 좌표계가 일치하는지 확인하고, 일치하지 않으면 변환
if gdf1.crs != gdf2.crs:
    gdf2 = gdf2.to_crs(gdf1.crs)
if gdf1.crs != gdf3.crs:
    gdf3 = gdf3.to_crs(gdf1.crs)

# GeoDataFrame 병합 (pd.concat 사용)
gdf_merged = pd.concat([gdf1, gdf2, gdf3], ignore_index=True)

# 병합된 데이터를 새로운 Shapefile로 저장
gdf_merged.to_file(output_shapefile)

print(f"병합된 Shapefile이 {output_shapefile}에 생성되었습니다.")

시도 중: euc-kr
시도 중: euc-kr
시도 중: euc-kr
병합된 Shapefile이 E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp에 생성되었습니다.


In [18]:
import geopandas as gpd

# "Jeju_Masked.shp" 파일 경로
masked_shp_file = r"E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp"

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(masked_shp_file)

# 'Mask' 필드의 모든 값을 0으로 설정
gdf['Mask'] = 0

# 수정된 GeoDataFrame을 다시 shapefile로 저장
gdf.to_file(masked_shp_file)

# 저장 완료 메시지 출력
print(f"{masked_shp_file}의 'Mask' 필드 값을 모두 0으로 설정했습니다.")


E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp의 'Mask' 필드 값을 모두 0으로 설정했습니다.


In [2]:
#Pvgen shp 배정
import os
import re
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

# 특수문자 제거 함수
def clean_filename(filename):
    cleaned_filename = re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)  # 특수문자는 '_'로 대체
    return cleaned_filename

# 입력 폴더 경로 및 출력 폴더 경로 설정
input_raster_folder = r"E:\2025_Jeju_SectorCoupling\PVgen"
shp_file = r"E:\2025_Jeju_SectorCoupling\LSMD_ADM_SECT_UMD_50_202503.shp"  # shp 파일 경로
output_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"

# 출력 폴더 생성 (없으면)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(shp_file)

# 폴더 내 모든 tif 파일 순회
for raster_file in os.listdir(input_raster_folder):
    if raster_file.endswith(".tif"):
        # 특수문자 제거한 파일명
        cleaned_raster_file = clean_filename(raster_file)
        
        raster_path = os.path.join(input_raster_folder, raster_file)

        # 래스터 파일 읽기
        with rasterio.open(raster_path) as src:
            # 래스터의 좌표계
            raster_crs = src.crs

            # 좌표계가 다르면 shp 파일 좌표계로 변환 (CRS 변환)
            if gdf.crs != raster_crs:
                gdf = gdf.to_crs(raster_crs)

            # 폴리곤에 해당하는 마스크 처리
            for _, row in gdf.iterrows():
                polygon = row['geometry']
                geo = [mapping(polygon)]

                # 마스크를 사용하여 폴리곤 영역만 추출
                out_image, out_transform = mask(src, geo, crop=True)

                # NoData 값이 있는 경우 NoData를 0으로 처리
                if src.nodata is not None:
                    out_image = np.ma.masked_equal(out_image, src.nodata)  # NoData 마스킹

                # 마스크된 이미지가 비어있거나, NoData 값만 포함된 경우 처리
                if out_image.sum() == 0:
                    total_sum = 0  # NoData만 있을 경우 0 처리
                else:
                    # numpy의 float64 타입으로 변환하여 합산
                    out_image = out_image.astype(np.float64)
                    total_sum = np.sum(out_image)  # 마스크된 영역의 총합 계산

                # 'pvgen' 값 업데이트
                gdf.at[_, 'pvgen'] = total_sum

        # 결과를 새로운 shp 파일로 저장
        output_shp = os.path.join(output_folder, f"{os.path.splitext(cleaned_raster_file)[0]}_Polygon.shp")
        gdf.to_file(output_shp)

        # 저장 완료 메시지 출력
        print(f"{output_shp} 저장 완료했습니다.")

# 종료 메시지
print("모든 래스터 파일에 대해 연산을 완료했습니다.")



E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_09-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_10-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_11-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_12-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_13-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_14-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_15-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_16-00-00_09-00_Polygon.shp 저장 완료했습니다.
E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen

KeyboardInterrupt: 

In [21]:
#발전배제지역추가버전
import os
import re
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

# 특수문자 제거 함수
def clean_filename(filename):
    return re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)  # 특수문자는 '_'로 대체

# 입력 폴더 경로 및 출력 폴더 경로 설정
input_raster_folder = r"E:\2025_Jeju_SectorCoupling\PVgen"
shp_file = r"E:\2025_Jeju_SectorCoupling\LSMD_ADM_SECT_UMD_50_202503.shp"  # shp 파일 경로
masked_shp_file = r"E:\2025_Jeju_SectorCoupling\Jeju_Masked.shp"  # Masking을 위한 shp 파일
output_folder = r"E:\2025_Jeju_SectorCoupling\PVgen_polygon"

# 출력 폴더 생성 (없으면)
os.makedirs(output_folder, exist_ok=True)

# GeoPandas로 shp 파일 읽기
gdf = gpd.read_file(shp_file)
masked_gdf = gpd.read_file(masked_shp_file)

# 'pvgen' 필드 추가
if 'pvgen' in gdf.columns:
    gdf = gdf.drop(columns=['pvgen'])
gdf['pvgen'] = 0.0  # 기본값 0으로 설정

# 폴더 내 모든 tif 파일 순회
for raster_file in os.listdir(input_raster_folder):
    if raster_file.endswith(".tif"):
        cleaned_raster_file = clean_filename(raster_file)
        raster_path = os.path.join(input_raster_folder, raster_file)

        # 래스터 파일 읽기
        with rasterio.open(raster_path) as src:
            # 래스터의 좌표계
            raster_crs = src.crs

            # 좌표계가 다르면 shp 파일 좌표계로 변환
            if gdf.crs != raster_crs:
                gdf = gdf.to_crs(raster_crs)

            if masked_gdf.crs != raster_crs:
                masked_gdf = masked_gdf.to_crs(raster_crs)

            # 먼저 Jeju_Masked.shp 파일의 모든 폴리곤에 대해 마스크 처리 (겹치는 영역을 0으로 설정)
            all_polygons = [mapping(polygon) for polygon in masked_gdf['geometry']]
            out_image, out_transform = mask(src, all_polygons, crop=True)

            # NoData 값이 있는 경우 NoData를 0으로 처리
            if src.nodata is not None:
                out_image = np.ma.masked_equal(out_image, src.nodata)

            # NoData 영역을 0으로 채움
            out_image = np.ma.filled(out_image, 0)

            # 나머지 영역 처리: pvgen 계산을 위한 합산 작업
            for idx, row in gdf.iterrows():
                polygon = row['geometry']
                geo = [mapping(polygon)]  # 폴리곤을 GeoJSON 형식으로 변환

                # 폴리곤에 해당하는 마스크 처리
                masked_image, _ = mask(src, geo, crop=True)

                # NoData 값이 있는 경우 NoData를 0으로 처리
                if src.nodata is not None:
                    masked_image = np.ma.masked_equal(masked_image, src.nodata)

                # 마스크된 이미지가 비어있거나, NoData 값만 포함된 경우 처리
                if masked_image.sum() == 0:
                    total_sum = 0  # NoData만 있을 경우 0 처리
                else:
                    masked_image = masked_image.astype(np.float64)
                    total_sum = np.sum(masked_image)

                # 'pvgen' 값 업데이트
                gdf.at[idx, 'pvgen'] = total_sum  # 적절한 인덱스에서 'pvgen' 값을 업데이트

        # 결과를 새로운 shp 파일로 저장
        output_shp = os.path.join(output_folder, f"{os.path.splitext(cleaned_raster_file)[0]}_Polygon.shp")
        gdf.to_file(output_shp)

        # 저장 완료 메시지 출력
        print(f"{output_shp} 저장 완료했습니다.")

# 종료 메시지
print("모든 래스터 파일에 대해 연산을 완료했습니다.")



E:\2025_Jeju_SectorCoupling\PVgen_polygon\TotalSolargen_2024-01-01_08-00-00_09-00_Polygon.shp 저장 완료했습니다.


KeyboardInterrupt: 